<a href="https://colab.research.google.com/github/DyonneSoares/Split_PDF_by_Text/blob/main/splitpdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalação das bibliotecas que não são nativas no Google Colaboraatory
!pip install PyPDF2
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Importando as bibliotecas
import os
import pdfplumber
import PyPDF2
from PyPDF2 import PdfWriter, PdfReader
from google.colab import drive

#Conectando o colab ao Google Drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def walklevel(dir, level=1):
  dir = dir.rstrip(os.path.sep)
  assert os.path.isdir(dir)
  num_sep = dir.count(os.path.sep)
  
  for root, dirs, files in os.walk(dir):
    yield root, dirs, files
    num_sep_this = root.count(os.path.sep)

    if num_sep + level <= num_sep_this:
      del dirs[:] 

In [ ]:
dir_files = '/content/gdrive/MyDrive/Postos/'

"#Caminho para o arquivo original\n#pdf_file = '/content/abril_SAMDU_2018_1.pdf'\n\ndir_files = '/content/gdrive/MyDrive/Postos/SAMDU/LMC 2017 SAMDU/'\npdf_file = '/content/gdrive/MyDrive/Postos/SAMDU/LMC 2017 SAMDU/dezembro_2017_SAMDU_2.pdf'\npdf_name = pdf_file.split('.')[0].split('/')[-1]"

In [ ]:
def processarPdf (arquivo):
  ### TEMPLATE PARA SAMDU 2019 A 2022 ###
  #Caminho para o arquivo original
  #pdf_file = '/content/abril_SAMDU_2018_1.pdf'

  #dir_files = '/content/gdrive/MyDrive/Postos/SAMDU/LMC_SAMDU_2019/'
  #_pdf_file  = '/content/gdrive/MyDrive/Postos/SAMDU/LMC_SAMDU_2019/DEZ.pdf'
  _pdf_file  = arquivo
  #print(_pdf_file)
  #_pdf_file_tratados = _pdf_file.replace('/Postos/', '/Postos/Tratados/')

  #if not _pdf_file_tratados :
  #Nome do pdf
  _pdf_name  = _pdf_file.split('.')[0].split('/')[-1]
  #Diretorio raiz
  _dir_files = arquivo.replace(_pdf_name+'.pdf','')

  #Muda o diretório para um específico
  _out_dir = _dir_files.replace('/Postos/', '/Postos/Tratados/')
  #Cria os diretórios
  #file2pages = {}
  os.makedirs(_out_dir,exist_ok = True)

  #Faz a leitura usando a biblioteca
  read_pdf = PyPDF2.PdfReader(open(_pdf_file, 'rb'))

  # pega o numero de páginas
  number_of_pages = len(read_pdf.pages)

  print('\n'+ _pdf_file)
  n_page = 0
  file2pages = {}
  produto = ''
  while n_page < number_of_pages:
    page = read_pdf.pages[n_page]
    #extrai apenas o texto
    page_content = page.extract_text()
    # faz a junção das linhas 
    parsed = ''.join(page_content)
    linhas = parsed.splitlines() 
    
    #Pega a página 2
    if 'Produto' not in parsed:
      file2pages[produto].append(n_page)
    else:
      for linha in linhas:
        #print(linha)
        if '1 Produto' in linha:
          produto = linha.replace('1 Produto', '').replace(':', '').replace(' ', '').replace('/', '')
          if produto not in file2pages:
            file2pages[produto] = [n_page]
          else:
            file2pages[produto].append(n_page)
        elif 'Produto' in linha:
          produto = linha.replace('Produto', '').replace(':', '').replace(' ', '').replace('/', '')[13:-1]
          if produto not in file2pages:
            file2pages[produto] = [n_page]
          else:
            file2pages[produto].append(n_page)

    #Incrementa para pegar o valor "real" da página
    n_page = n_page + 1

  #Abre o pdf original no modo de leitura
  with open(_pdf_file, 'rb') as pdf:
    #Cria o objeto PdfReader para leitura 
    pdf_content = PdfReader(_pdf_file)
    
    #Armazena o quantidade de páginas do pdf original
    for t_combustivel in file2pages:
      #Cria o objeto PdfWriter, para escrita
      pdf_writer = PdfWriter()
      for page in file2pages[t_combustivel]:
        pdf_writer.add_page(pdf_content.pages[page])
        
        #Invoca a função pdf_get_name para extrair o nome contido na página atual
        pdf_name_tipo = _pdf_name + "____" + t_combustivel
        #O médoto os.path.join() une o caminho para gravação, o nome e a extesão do arquivo pdf. 
        pdf_out = os.path.join(_out_dir, pdf_name_tipo +'.pdf')
        
        #Grava o objeto de escrita no arquivo
        with open(pdf_out, 'wb') as pdf_named:
          pdf_writer.write(pdf_named)


In [ ]:
for pasta, subpastas, arquivos in walklevel(dir_files,level=0):
#for pasta, subpastas, arquivos in os.walk(path, 0):
  path_tratados = dir_files+"Tratados/"
  
  for subpasta in subpastas:
    if 'Tratados' != subpasta:
      pasta_nivel_1 = dir_files+subpasta
      for pasta, subpastas, arquivos in walklevel(pasta_nivel_1,level=0):
      #for pasta, subpastas, arquivos in os.walk(pasta_nivel_1):
        for subpasta in subpastas:
          pasta_nivel_2 = pasta_nivel_1+"/"+subpasta
          for pasta, subpastas, arquivos in walklevel(pasta_nivel_2,level=0):
          #for pasta, subpastas, arquivos in os.walk(pasta_nivel_2):
            for arquivo in arquivos:
              processarPdf(pasta_nivel_2+'/'+arquivo)



/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/agosto_2017_EPTG_1.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/agosto_2017_EPTG_2.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/agosto_2017_EPTG_3.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/dezembro_2017_EPTG_1.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/dezembro_2017_EPTG_2.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/dezembro_2017_EPTG_3.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/setembro_2017_EPTG_1.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/setembro_2017_EPTG_2.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/setembro_2017_EPTG_3.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/outubro_2017_EPTG_1.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/outubro_2017_EPTG_2.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/outubro_2017_EPTG_3.pdf

/content/gdrive/MyDrive/Postos/EPTG/LMC 2017 EPTG/novembro_2017_EPTG_1.pdf

/content/gdrive/MyDr